In [1]:
import numpy as np

1. Considere:

$$
\begin{align*}
    \begin{bmatrix}
        1 & 2 & 3\\
        3 & 2 & 1\\
        1 & 1 & -1
    \end{bmatrix}
    \mathbf{x} &=
    \begin{bmatrix}
        1\\
        1\\
        1
    \end{bmatrix}.
\end{align*}
$$

a. Construya el sub-espacio de Krylov *no-ortonormalizado* asociado.

b. Construya el sub-espacio de Krylov *ortonormalizado* asociado.

c. Construya la descomposición de Hessenberg asociada.

d. Determine el sistema de ecuaciones *equivalente* a resolver con GMRes.

In [2]:
#Definimos la matriz y el lado derecho del sistema Ax = b
A = np.array([[1.,2.,3.],[3.,2.,1.],[1.,1.,-1.]])
b = np.array([1.,1.,1.])

a. El sub-espacio de Krylov viene dado por:

$$
K_k = \{\mathbf{b},A\,\mathbf{b}, A^2\,\mathbf{b}, \dots, A^{k-1}\,\mathbf{b}\}
$$

Para este caso $k = 3$:

$$
K_3 = \{\mathbf{b},A\,\mathbf{b}, A^2\,\mathbf{b}\} = \{\mathbf{b}_1,\mathbf{b}_2,\mathbf{b}_3\}
$$

In [9]:
#sub-espacio de Krylov no-ortonormalizado
b1 = b
b2 = np.dot(A,b1)
b3 = np.dot(A,b2)

print("b1")
print(b1[:,None])
print("b2")
print(b2[:,None])
print("b3")
print(b3[:,None])

b1
[[1.]
 [1.]
 [1.]]
b2
[[6.]
 [6.]
 [1.]]
b3
[[21.]
 [31.]
 [11.]]


b. Para construir el sub-espacio de Krylov ortonormalizado, aplicamos la iteración de Arnoldi, con lo cual obtendremos los coeficientes $h_{ij}$ y los vectores $q_i$.

Iteración $1$:

$\mathbf{q}_1 = \mathbf{b} / \lVert \mathbf{b} \rVert$

$A\,\mathbf{q}_1 = h_{11}\,\mathbf{q}_1 + h_{21}\,\mathbf{q}_2$

In [27]:
#Se obtiene q1
q1 = b / np.linalg.norm(b)
print("q1")
print(q1[:,None])

q1
[[0.57735027]
 [0.57735027]
 [0.57735027]]


Multiplicamos $\mathbf{q}_1^*$ a la ecuación:

$\mathbf{q}_1^*(A\,\mathbf{q}_1) = h_{11}$

Restamos $h_{11}\,\mathbf{q}_1$:

$A\,\mathbf{q}_1 - h_{11}\,\mathbf{q}_1 = h_{21}\,\mathbf{q}_2$

Aplicamos norma:

$\lVert A\,\mathbf{q}_1 - h_{11}\,\mathbf{q}_1 \rVert = h_{21}$

In [28]:
#Se obtiene h11 y h21
y = np.dot(A,q1)
h11 = np.dot(q1,y)
y = y - h11*q1
h21 = np.linalg.norm(y)

print("h11:",h11)
print("h21:",h21)

h11: 4.333333333333334
h21: 2.357022603955159


Iteración 2:

Obtenemos $\mathbf{q}_2$:

$\mathbf{q}_2 = (A\,\mathbf{q}_1 - h_{11}\,\mathbf{q}_1) / h_{21}$

La ecuación para $A\,\mathbf{q}_2$ es:

$A\,\mathbf{q}_2 = h_{12}\,\mathbf{q}_1 + h_{22}\,\mathbf{q}_2 + h_{32}\,\mathbf{q}_3$

Multiplicamos $\mathbf{q}_1^*$ a la ecuación:

$\mathbf{q}_1^*(A\,\mathbf{q}_2) = h_{12}$

Multiplicamos $\mathbf{q}_2^*$ a la ecuación:

$\mathbf{q}_2^*(A\,\mathbf{q}_2 - h_{12}\,\mathbf{q}_1) = h_{22}$

Restamos $h_{12}\,\mathbf{q}_1 + h_{22}\,\mathbf{q}_2$:

$A\,\mathbf{q}_2 - h_{12}\,\mathbf{q}_1 - h_{22}\,\mathbf{q}_2 = h_{32}\,\mathbf{q}_3$

In [29]:
#Obtenemos q2
q2 = y / h21

#Obtenemos h12, h22 y h32
y = np.dot(A,q2)
h12 = np.dot(q1,y)
y = y - h12*q1
h22 = np.dot(q2,y)
y = y - h22*q2
h32 = np.linalg.norm(y)

print("q2")
print(q2[:,None])

print("h12:",h12)
print("h22:",h22)
print("h32:",h32)

q2
[[ 0.40824829]
 [ 0.40824829]
 [-0.81649658]]
h12: 0.9428090415820629
h22: -1.333333333333334
h32: 1.7320508075688776


Finalmente, calculamos $\mathbf{q}_3$

$\mathbf{q}_3 = (A\,\mathbf{q}_2 - h_{12}\,\mathbf{q}_1 - h_{22}\,\mathbf{q}_2) / h_{32}$

In [30]:
q3 = y / h32

print("q3")
print(q3[:,None])

q3
[[-7.07106781e-01]
 [ 7.07106781e-01]
 [-2.56395025e-16]]


Como $h_{32} > 0$, significa que aún podemos realizar una iteración más:

La ecuación para $A\,\mathbf{q}_3$ es:

$A\,\mathbf{q}_3 = h_{13}\,\mathbf{q}_1 + h_{23}\,\mathbf{q}_2 + h_{33}\,\mathbf{q}_3 + h_{43}\,\mathbf{q}_4$

Multiplicamos $\mathbf{q}_1^*$ a la ecuación:

$\mathbf{q}_1^*(A\,\mathbf{q}_3) = h_{13}$

Multiplicamos $\mathbf{q}_2^*$ a la ecuación:

$\mathbf{q}_2^*(A\,\mathbf{q}_3 - h_{13}\,\mathbf{q}_1) = h_{23}$

Multiplicamos $\mathbf{q}_3^*$ a la ecuación:

$\mathbf{q}_2^*(A\,\mathbf{q}_3 - h_{13}\,\mathbf{q}_1 - h_{23}\,\mathbf{q}_2) = h_{33}$

Restamos $h_{13}\,\mathbf{q}_1 + h_{23}\,\mathbf{q}_2 + h_{33}\,\mathbf{q}_3$:

$A\,\mathbf{q}_3 - h_{13}\,\mathbf{q}_1 - h_{23}\,\mathbf{q}_2 - h_{33}\,\mathbf{q}_3$

Aplicamos norma:

$\lVert A\,\mathbf{q}_3 - h_{13}\,\mathbf{q}_1 - h_{23}\,\mathbf{q}_2 - h_{33}\,\mathbf{q}_3 \rVert = h_{43}$

In [31]:
y = np.dot(A,q3)
h13 = np.dot(q1,y)
y = y - h13*q1
h23 = np.dot(q2,y)
y = y - h23*q2
h33 = np.dot(q3,y)
y = y - h33*q3
h43 = np.linalg.norm(y)

print("h13:",h13)
print("h23:",h23)
print("h33:",h33)
print("h43:",h43)

h13: -9.992007221626409e-16
h23: -8.326672684688674e-16
h33: -0.9999999999999999
h43: 2.720397184653305e-16


Podemos observar que $h_{43} \approx 0$ por lo tanto dejamos de iterar.

c. Construya la descomposición de Hessenberg asociada.

Como hemos dejado de iterar, construimos la descomposición:

$A\,Q_3 = Q_4\,H_3$

Notar que en este caso $Q_3 = Q_4$

In [34]:
Q = np.zeros((3,3))
Q[:,0] = q1
Q[:,1] = q2
Q[:,2] = q3

H3 = np.array([[h11,h12,h13],[h21,h22,h23],[0.,h32,h33]])

AQ = A@Q
QH3 = Q@H3

#Comprobamos que el lado izquierdo y derecho son iguales
print("A x Q3")
print(AQ)
print("Q4 x H3")
print(QH3)

A x Q3
[[ 3.46410162e+00 -1.22474487e+00  7.07106781e-01]
 [ 3.46410162e+00  1.22474487e+00 -7.07106781e-01]
 [ 5.77350269e-01  1.63299316e+00  1.45372722e-16]]
Q4 x H3
[[ 3.46410162e+00 -1.22474487e+00  7.07106781e-01]
 [ 3.46410162e+00  1.22474487e+00 -7.07106781e-01]
 [ 5.77350269e-01  1.63299316e+00  3.59376197e-16]]


d. Luego, el sistema equivalente es:

$H_3\,\mathbf{c} = \lVert \mathbf{b}\rVert\,\mathbf{e}_1$

$H_3\,\begin{bmatrix}c_1 \\ c_2 \\ c_3\end{bmatrix} = \begin{bmatrix}\lVert \mathbf{b} \rVert \\ 0 \\ 0\end{bmatrix}$

$Qh,Rh = qr(H_3)$

$Rh\,\mathbf{c} = Qh^*\,\lVert \mathbf{b}\rVert\,\mathbf{e}_1$

In [38]:
#Calculamos el vector de coeficientes.
be = np.array([np.linalg.norm(b),0.,0.])
Qh,Rh = np.linalg.qr(H3,mode="reduced")
bh = np.dot(Qh.T,be)
c = np.linalg.solve(Rh,bh)
print("c:")
print(c[:,None])

c:
[[0.28867513]
 [0.51031036]
 [0.88388348]]


In [39]:
#Calculamos la solución
x = np.dot(Q,c)
print("x:")
print(x[:,None])

x:
[[-0.25]
 [ 1.  ]
 [-0.25]]


Comprobamos que efectivamente es la solución del sistema $A\,\mathbf{x} = \mathbf{b}$:

In [40]:
Ax = np.dot(A,x)
print("Ax:")
print(Ax[:,None])
print("b:")
print(b[:,None])

Ax:
[[1.]
 [1.]
 [1.]]
b:
[[1.]
 [1.]
 [1.]]
